In [ ]:
!gcloud dataproc clusters list --region us-central1
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
import itertools
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from pathlib import Path
import pickle
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
bucket_name = 'que_buck' 
client = storage.Client()

full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text", "id").rdd

%cd -q /home/dataproc
!ls inverted_index_gcp.py

sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

from inverted_index_gcp import InvertedIndex

In [ ]:
import math
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def doc_length(text,id1):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    d={}
    s=0
    for token in tokens:
        d[token]=d.get(token,0)+1
    for k,v in d.items():
        s+=v*v
    q=math.sqrt(s)
    return id1,q

lens=doc_text_pairs.map(lambda x:doc_length(x[0],x[1]))
dict_len=lens.collectAsMap()

with open("." / f'norm.pkl', 'wb') as f:
    pickle.dump(dict_len, f)

index_src = "norm.pkl"
index_dst = f'gs://{bucket_name}/pr/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
doc_title_pairs = parquetFile.select("title", "id").rdd
titles=doc_title_pairs.collectAsMap()

with open("." / f'titles.pkl', 'wb') as f:
    pickle.dump(dict_len, f)

index_src = "titles.pkl"
index_dst = f'gs://{bucket_name}/pr/{index_src}'
!gsutil cp $index_src $index_dst